<a href="https://colab.research.google.com/github/veydantkatyal/ai-trading-bot/blob/main/botwise_RL_Portfolio_optmization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INSTALL DEPENDENCIES**

In [ ]:
!pip install stable-baselines3[extra] gym gym-anytrading pandas numpy matplotlib seaborn ccxt yfinance textblob vaderSentiment backtrader scipy flask fastapi uvicorn requests streamlit tensorflow torch transformers cvxpy optuna plotly shimmy>=2.0 stable-baselines3 gym numpy pandas


# **FETCH LIVE MARKET DATA**

**Cryptocurrency Prices (Using CoinGecko API)**

In [ ]:
import requests

crypto_symbols = ["ethereum", "bitcoin", "binancecoin", "ripple", "solana",
                  "cardano", "dogecoin", "polkadot", "avalanche-2", "chainlink"]

def fetch_crypto_prices(symbols=crypto_symbols):
    url = f"https://api.coingecko.com/api/v3/simple/price?ids={','.join(symbols)}&vs_currencies=usd"
    response = requests.get(url).json()
    return {symbol: response[symbol]["usd"] for symbol in symbols}

# Test
crypto_prices = fetch_crypto_prices()
print("Crypto Prices:", crypto_prices)


Crypto Prices: {'ethereum': 2708.72, 'bitcoin': 97774, 'binancecoin': 661.75, 'ripple': 2.81, 'solana': 195.31, 'cardano': 0.79293, 'dogecoin': 0.278465, 'polkadot': 5.1, 'avalanche-2': 26.12, 'chainlink': 19.17}


**Stock Prices (Using Yahoo Finance)**

In [ ]:
import yfinance as yf

stock_symbols = ["AAPL", "TSLA", "GOOGL", "AMZN", "MSFT", "META", "NVDA", "NFLX", "INTC", "IBM"]

def fetch_stock_prices(symbols=stock_symbols):
    prices = {}
    for symbol in symbols:
        stock = yf.Ticker(symbol)
        prices[symbol] = stock.history(period="1d", interval="1h")['Close'].iloc[-1]
    return prices

# Test
stock_prices = fetch_stock_prices()
print("Stock Prices:", stock_prices)


Stock Prices: {'AAPL': 244.57000732421875, 'TSLA': 355.8999938964844, 'GOOGL': 185.1699981689453, 'AMZN': 228.67999267578125, 'MSFT': 408.3299865722656, 'META': 736.5999755859375, 'NVDA': 138.86000061035156, 'NFLX': 1058.780029296875, 'INTC': 23.610000610351562, 'IBM': 261.2099914550781}


**Forex Prices (Using Alpha Vantage API)**

In [ ]:
import requests

# Define Forex currency pairs
forex_pairs = [("EUR", "USD"), ("GBP", "USD"), ("USD", "JPY"), ("AUD", "USD"), ("USD", "CAD"),
               ("USD", "CHF"), ("NZD", "USD"), ("EUR", "GBP"), ("EUR", "JPY"), ("GBP", "JPY")]

EXCHANGERATE_API_KEY = "YOUR_API_KEY"
FOREX_URL = f"https://v6.exchangerate-api.com/v6/{EXCHANGERATE_API_KEY}/latest/"

def fetch_forex_prices(pairs=forex_pairs):
    prices = {}
    for from_currency, to_currency in pairs:
        try:
            url = f"{FOREX_URL}{from_currency}"
            response = requests.get(url).json()
            rates = response.get("conversion_rates", {})
            if to_currency in rates:
                prices[f"{from_currency}/{to_currency}"] = rates[to_currency]
            else:
                prices[f"{from_currency}/{to_currency}"] = None
                print(f"Error fetching {from_currency}/{to_currency}: No data found")
        except Exception as e:
            prices[f"{from_currency}/{to_currency}"] = None
            print(f"Error fetching {from_currency}/{to_currency}: {e}")

    return prices

# Test
forex_prices = fetch_forex_prices()
print("Forex Prices:", forex_prices)


Forex Prices: {'EUR/USD': 1.0492, 'GBP/USD': 1.2585, 'USD/JPY': 152.3849, 'AUD/USD': 0.6349, 'USD/CAD': 1.4176, 'USD/CHF': 0.8996, 'NZD/USD': 0.5724, 'EUR/GBP': 0.8332, 'EUR/JPY': 159.7919, 'GBP/JPY': 191.8276}


**Commodity Prices (Using Yahoo Finance)**

In [ ]:
import yfinance as yf

# Define commodity tickers from Yahoo Finance
commodity_tickers = {
    "Gold": "GC=F",
    "Silver": "SI=F",
    "Platinum": "PL=F",
    "Palladium": "PA=F",
    "Crude Oil WTI": "CL=F",
    "Crude Oil Brent": "BZ=F",
    "Natural Gas": "NG=F",
    "Copper": "HG=F",
    "Aluminum": "ALI=F"
}

def fetch_commodity_prices():
    prices = {}
    for name, ticker in commodity_tickers.items():
        try:
            data = yf.Ticker(ticker)
            hist = data.history(period="1d")
            if not hist.empty:
                prices[name] = hist['Close'].iloc[-1]
            else:
                prices[name] = None
                print(f"No data found for {name}")
        except Exception as e:
            prices[name] = None
            print(f"Error fetching {name}: {e}")
    return prices

# Test
commodity_prices = fetch_commodity_prices()
print("Updated Commodity Prices:", commodity_prices)


Updated Commodity Prices: {'Gold': 2883.60009765625, 'Silver': 32.80099868774414, 'Platinum': 1009.7999877929688, 'Palladium': 1005.0, 'Crude Oil WTI': 70.73999786376953, 'Crude Oil Brent': 74.73999786376953, 'Natural Gas': 3.7249999046325684, 'Copper': 4.656499862670898, 'Aluminum': 2612.25}


**Consolidate All Asset Prices**

In [ ]:
assets = {
    "Crypto": fetch_crypto_prices(),
    "Stocks": fetch_stock_prices(),
    "Forex": fetch_forex_prices(),
    "Commodities": fetch_commodity_prices()
}

print("All Asset Prices:", assets)


All Asset Prices: {'Crypto': {'ethereum': 2708.72, 'bitcoin': 97774, 'binancecoin': 661.75, 'ripple': 2.81, 'solana': 195.31, 'cardano': 0.79293, 'dogecoin': 0.278465, 'polkadot': 5.1, 'avalanche-2': 26.12, 'chainlink': 19.17}, 'Stocks': {'AAPL': 244.57000732421875, 'TSLA': 355.8999938964844, 'GOOGL': 185.1699981689453, 'AMZN': 228.67999267578125, 'MSFT': 408.3299865722656, 'META': 736.5999755859375, 'NVDA': 138.86000061035156, 'NFLX': 1058.780029296875, 'INTC': 23.610000610351562, 'IBM': 261.2099914550781}, 'Forex': {'EUR/USD': 1.0492, 'GBP/USD': 1.2585, 'USD/JPY': 152.3849, 'AUD/USD': 0.6349, 'USD/CAD': 1.4176, 'USD/CHF': 0.8996, 'NZD/USD': 0.5724, 'EUR/GBP': 0.8332, 'EUR/JPY': 159.7919, 'GBP/JPY': 191.8276}, 'Commodities': {'Gold': 2883.60009765625, 'Silver': 32.80099868774414, 'Platinum': 1009.7999877929688, 'Palladium': 1005.0, 'Crude Oil WTI': 70.73999786376953, 'Crude Oil Brent': 74.73999786376953, 'Natural Gas': 3.7249999046325684, 'Copper': 4.656499862670898, 'Aluminum': 2612.

# **AI-Based Portfolio Optimization**

* Markowitz Mean-Variance Optimization (Classic Portfolio Theory)
* Reinforcement Learning-Based Portfolio Optimization (Deep Q-Learning)

**Markowitz Approach**

This method finds the optimal asset allocation that maximizes return while minimizing risk.

In [ ]:
import numpy as np
import cvxpy as cp

# Simulated historical returns for each asset class
returns = np.array([
    [0.01, 0.02, 0.015, 0.005, -0.002],  # Crypto
    [0.012, 0.018, 0.01, 0.008, 0.002],  # Stocks
    [0.007, 0.011, 0.009, 0.006, 0.001],  # Forex
    [0.005, 0.007, 0.006, 0.003, -0.001]  # Commodities
]).T

# Number of assets
num_assets = returns.shape[1]

# Define optimization variables
weights = cp.Variable(num_assets)

# Calculate expected returns (you can replace this with your desired method)
expected_return = np.mean(returns, axis=0)

# Define the expected_return parameter and assign the calculated values
expected_return_param = cp.Parameter(num_assets, nonneg=True, value=expected_return)

# Calculate covariance matrix
cov_matrix = np.cov(returns.T)

# Use the expected_return_param in the objective function
objective = cp.Maximize(expected_return_param @ weights - 0.5 * cp.quad_form(weights, cov_matrix))
constraints = [cp.sum(weights) == 1, weights >= 0]

# Solve optimization problem
problem = cp.Problem(objective, constraints)
problem.solve()

# Display Optimized Portfolio Weights
optimized_weights = weights.value
print("Optimized Portfolio Weights:", optimized_weights)

Optimized Portfolio Weights: [-1.73464598e-22  1.00000000e+00 -3.03440701e-27 -4.41602409e-27]


**RL Approach**

We use Deep Deterministic Policy Gradient(DDPG) to dynamically allocate assets based on market conditions.

In [ ]:
import gym
import numpy as np
#from gym import spaces  # Original import
from gymnasium import spaces  # Import from gymnasium instead of gym
# Import necessary modules for compatibility
import shimmy
# Correct import statement for gymnasium
import gymnasium as gym

class PortfolioEnv(gym.Env):
    def __init__(self, asset_prices, initial_balance=10000):
        super(PortfolioEnv, self).__init__()

        self.asset_prices = asset_prices
        self.num_assets = asset_prices.shape[1]
        self.initial_balance = initial_balance
        self.current_balance = initial_balance
        self.current_step = 0

        # Action space: Allocate % of portfolio to each asset
        self.action_space = spaces.Box(low=0, high=1, shape=(self.num_assets,), dtype=np.float32)

        # Observation space: Asset prices and balance
        self.observation_space = spaces.Box(low=0, high=np.inf, shape=(self.num_assets + 1,), dtype=np.float32)

    def reset(self, seed=None, options=None): # Add seed and options argument
        super().reset(seed=seed) # Pass seed to super().reset()
        self.current_balance = self.initial_balance
        self.current_step = 0
        return self._next_observation(), {} # Return info dict

    def _next_observation(self):
        obs = np.append(self.asset_prices[self.current_step], self.current_balance)
        return obs

    def step(self, action):
        allocation = action / np.sum(action)  # Normalize allocation
        self.current_balance *= 1 + np.dot(allocation, self.asset_prices[self.current_step] / self.asset_prices[self.current_step - 1] - 1)
        self.current_step += 1

        done = self.current_step >= len(self.asset_prices) - 1
        # Return observation, reward, terminated, truncated, info
        # In this case, we'll set terminated to done and truncated to False
        return self._next_observation(), self.current_balance, done, False, {}

    def render(self):
        print(f"Step: {self.current_step}, Balance: {self.current_balance}")

In [ ]:
from stable_baselines3 import DQN, DDPG

# Simulated Asset Prices for Training (Replace with real data)
np.random.seed(42)
asset_prices = np.random.rand(1000, 4) * 100  # 1000 time steps, 4 assets

# Create environment
env = PortfolioEnv(asset_prices)

# Train DQN model
model = DDPG("MlpPolicy", env, learning_rate=0.0003, gamma=0.99, batch_size=64, verbose=1)
model.learn(total_timesteps=500000)
model.save("ddpg_portfolio_optimizer")

print("Reinforcement Learning Portfolio Optimization Model Trained Successfully!")


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 999      |
|    ep_rew_mean     | nan      |
| time/              |          |
|    episodes        | 4        |
|    fps             | 155      |
|    time_elapsed    | 25       |
|    total_timesteps | 3996     |
| train/             |          |
|    actor_loss      | nan      |
|    critic_loss     | nan      |
|    learning_rate   | 0.0003   |
|    n_updates       | 3895     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 999      |
|    ep_rew_mean     | nan      |
| time/              |          |
|    episodes        | 8        |
|    fps             | 155      |
|    time_elapsed    | 51       |
|    total_timesteps | 7992     |
| train/             |          |
|    actor_loss      | nan      |
|    critic_loss     